In [22]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tensorflow.keras import layers, models
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
train_df = pd.read_csv('drive/MyDrive/Colab Notebooks/train.csv')
test_df = pd.read_csv('drive/MyDrive/Colab Notebooks/test.csv')

train data

In [23]:
train_y = train_df.iloc[:, 0]
train_y = to_categorical(train_y, num_classes=5)
train_x = train_df.iloc[:, 1:]
train_x = train_x / 255.0
train_x = train_x.to_numpy()
num_of_images = len(train_x)
train_x = train_x.reshape((num_of_images, 28, 28))

test data

In [31]:
test_y = test_df.iloc[:, 0]
test_y = to_categorical(test_y, num_classes=5)
test_x = test_df.iloc[:, 1:]
test_x = test_x / 255.0
test_x = test_x.to_numpy()
num_of_test_images = len(test_x)
test_x = test_x.reshape((num_of_test_images, 28, 28))

In [25]:
# CNN
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
history = model.fit(train_x, train_y, epochs=50, validation_split=0.05)

Epoch 1/50
891/891 [==============================] - 10s 6ms/step - loss: 0.1597 - accuracy: 0.9507 - val_loss: 0.0903 - val_accuracy: 0.9753
Epoch 2/50
891/891 [==============================] - 4s 4ms/step - loss: 0.0744 - accuracy: 0.9755 - val_loss: 0.0648 - val_accuracy: 0.9800
Epoch 3/50
891/891 [==============================] - 4s 5ms/step - loss: 0.0609 - accuracy: 0.9807 - val_loss: 0.0690 - val_accuracy: 0.9773
Epoch 4/50
891/891 [==============================] - 5s 5ms/step - loss: 0.0513 - accuracy: 0.9839 - val_loss: 0.0614 - val_accuracy: 0.9833
Epoch 5/50
891/891 [==============================] - 4s 4ms/step - loss: 0.0446 - accuracy: 0.9846 - val_loss: 0.0521 - val_accuracy: 0.9833
Epoch 6/50
891/891 [==============================] - 4s 5ms/step - loss: 0.0380 - accuracy: 0.9866 - val_loss: 0.0562 - val_accuracy: 0.9840
Epoch 7/50
891/891 [==============================] - 5s 5ms/step - loss: 0.0331 - accuracy: 0.9885 - val_loss: 0.0642 - val_accuracy: 0.9847
Epoch

save model and weights

In [28]:
model_json = model.to_json()
with open('drive/MyDrive/robotic_model.json', 'w') as json_file:
  json_file.write(model_json)

model.save_weights('drive/MyDrive/robotic_weights.h5')

load model and weights

In [29]:
from tensorflow.keras.models import model_from_json

json_model = open('drive/MyDrive/robotic_model.json', 'r')
loaded_json_model = json_model.read()
json_model.close()
loaded_model = model_from_json(loaded_json_model)
loaded_model.load_weights('drive/MyDrive/robotic_weights.h5')

In [30]:
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
_, score = loaded_model.evaluate(test_x, test_y, verbose=0)
score

0.982200026512146